In [0]:
dbutils.widgets.text("data_execucao", "")
data_execucao = dbutils.widgets.get("data_execucao")

In [0]:
import requests
from pyspark.sql.functions import lit

In [0]:
def extracao(date, base='BRL'):

  url = f"https://api.apilayer.com/exchangerates_data/{date}&base={base}"

  payload = {}
  headers= {
    "apikey": "YOUR API KEY HERE"
  }

  parametros = {"base": base}

  response = requests.request("GET", url, headers=headers, params=parametros)

  if response.status_code != 200:
      raise Exception("Não foi possível extrair os dados!")

  return response.json()


In [0]:
def dados_para_dataframe(dados_json):
    dados_tupla = [(moeda, float(taxa)) for moeda, taxa in dados_json['rates'].items()]
    return dados_tupla

In [0]:
def salvar_parquet(conversoes_extraidas):
    ano, mes, dia = conversoes_extraidas['date'].split('-')
    path = f"dbfs:/databricks-results/bronze/{ano}/{mes}/{dia}"
    result = dados_para_dataframe(conversoes_extraidas)
    df_moedas = spark.createDataFrame(result, schema=['moeda', 'taxa'])
    df_moedas = df_moedas.withColumn('data', lit(f'{ano}-{mes}-{dia}'))

    df_moedas.write.format('parquet').mode('overwrite').save(path)
    print(f"Dados salvos em {path}")


In [0]:
cotacoes = extracao(data_execucao)
salvar_parquet(cotacoes)
